<a href="https://colab.research.google.com/github/rummannaqvi/uniquest/blob/master/UniQuestVectorStore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 70.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive', force_remount= True)

VECTORSTORE_PATH = "/content/drive/MyDrive/vectorstore/faiss_index"
DOCS_JSONL_PATH = "docs_with_metadata (2).jsonl"


Mounted at /content/drive


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json

embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-base-en",           # You can try others like "thenlper/gte-large"
    model_kwargs={"device": "cuda"}          # This tells it to use GPU
)

# Check if vectorstore already exists
if not os.path.exists(VECTORSTORE_PATH):
    print("No vectorstore found. Loading JSONL and building vectorstore...")

    # Load previously saved scraped data
    loaded_docs = []
    with open(DOCS_JSONL_PATH, "r", encoding="utf-8") as f:
        for line in f:
            item = json.loads(line)
            metadata = {k: item[k] for k in item if k != "content"}
            if "url" not in metadata and "url" in item:
              metadata["url"] = item["url"]

            loaded_docs.append(Document(page_content=item["content"], metadata=metadata))

    # Split text
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    split_docs = splitter.split_documents(loaded_docs)

    # Create vectorstore
    vectorstore = FAISS.from_documents(split_docs, embedding_model)

    # Save it for next time
    vectorstore.save_local(VECTORSTORE_PATH)
    print("Vectorstore created and saved.")


No vectorstore found. Loading JSONL and building vectorstore...
Vectorstore created and saved.
